# **Prueba de Matriz con Openrouteservice**

## Cargar modulos

In [2]:
import openrouteservice as ors
import folium
import geopandas as gpd

## Importar y procesar Shape de Capitales de Provincias

Descarga [aquí](https://www.geogpsperu.com/2016/10/capital-provincial-ign-inei-descargar.html)

In [3]:
cap_p = "D:/MTC_OGPPOE/04_Actividades/10_MatrizAPI/inputs/Cap_Provincia/Cap_Provincia.shp"
db_prov = gpd.read_file(cap_p)
db_LL = db_prov[db_prov.Departa=='LA LIBERTAD']
db_LL[['Y','X']].values.tolist()

[[-8.11179, -79.02511],
 [-8.04117, -78.48728],
 [-8.27638, -77.29612],
 [-8.14508, -78.17584],
 [-8.4157, -78.75177],
 [-7.71518, -79.10748],
 [-7.90168, -78.5643],
 [-7.81483, -78.05003],
 [-7.22772, -79.42914],
 [-7.43196, -79.50439],
 [-7.47826, -78.81812],
 [-7.15489, -77.70096]]

## Conectarse a _Openrouteservice_

Servicio [aquí](https://openrouteservice.org/). Es una versión free, por lo cual hay un número limitados de queries.

In [4]:
client = ors.Client(key='clave')

## Calcular matrices

In [10]:
# Coordenadas a calcular
coordinates = db_LL[['X','Y']].values.tolist()
cod_prov = db_LL['Cod_Provin'].values.tolist()
# Calcular matrix
matrix = client.distance_matrix(
    locations=coordinates,
    profile='driving-car',
    metrics=['distance', 'duration'],
    validate=False
)

## Construir mapa

In [11]:
m = folium.Map(location=[-8.11179, -79.02511], tiles='cartodbpositron', zoom_start=8)
for marker in coordinates:
    folium.Marker(location=list(reversed(marker))).add_to(m)
m

## Función para formatear matrices

In [12]:
def reporta_matriz(matrix, cod_prov, tipo):
    """
    Input:
        matrix: resultado de openstreet
        cod_prov: codigo de ubicaciones
        tipo: distancia o duracion
    Output:
        mt: matriz
    """
    
    if tipo not in ["distancia", "duracion"]:
        raise Exception("Argumento 'tipo' no autorizado")
    
    mt = pd.DataFrame(matrix['distances'])
    mt.columns = cod_prov; mt.index = cod_prov; mt['origen'] = mt.index
    mt = pd.melt(mt, id_vars = 'origen', var_name = 'destino', value_name = 'values')
    mt['tipo'] = tipo
    
    if tipo == 'distancia':
        mt['unidad_medida'] = 'metros'
    else:
        mt['unidad_medida'] = 'segundos'
    
    return mt

In [13]:
mat_duracion = reporta_matriz(matrix = matrix, cod_prov = cod_prov, tipo = 'duracion')
mat_distancia = reporta_matriz(matrix = matrix, cod_prov = cod_prov, tipo = 'distancia')

In [15]:
mat_duracion.head()

,origen,destino,values,tipo,unidad_medida
0,1301,1301,0.00,duracion,segundos
1,1305,1301,106188.02,duracion,segundos
2,1308,1301,437050.75,duracion,segundos
3,1310,1301,171954.34,duracion,segundos
4,1312,1301,50471.21,duracion,segundos


In [16]:
mat_distancia.head()

,origen,destino,values,tipo,unidad_medida
0,1301,1301,0.00,distancia,metros
1,1305,1301,106188.02,distancia,metros
2,1308,1301,437050.75,distancia,metros
3,1310,1301,171954.34,distancia,metros
4,1312,1301,50471.21,distancia,metros


Si verificamos los resultados con [google maps](https://www.google.com.pe/maps/@-12.0630149,-77.0296179,13z?hl=es-419), las cifras son muy similares :)